# Trash Classification

This notebook is detecated to find out if a given trash (via its image) is recyclable or not. We'll try to adapt the previous model to the torch library


## Importing modules

In [45]:
import pandas as pd
import numpy as np
import os

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

## Loading Data

### Defining a function to retrieve data (downloaded from Kaggle)

In [40]:
class TrashDataset(Dataset):
    def __init__(self, dir_path, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform
        class_map = {'R': 0, 'O': 1}

        for label_folder in ['R', 'O']:
            folder_path = os.path.join(dir_path, label_folder)
            for file in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, file))
                self.labels.append(class_map[label_folder])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [41]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

### CNN Model (equivalent to Sequential)

In [42]:
class TrashCNN(nn.Module):
    def __init__(self):
        super(TrashCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # Pour binaire : 1 neurone

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x


### Model Training

#### Preparing Data

In [43]:
TRAIN_PATH = "data/TRAIN"
TEST_PATH = "data/TEST"

In [48]:
dataset = TrashDataset(TRAIN_PATH, transform=transform)
train_len = int(0.8*len(dataset))
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len])

In [49]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

#### Initializing the model

In [52]:
model = TrashCNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#### Training

In [54]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        labels = labels.float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculating accuracy
        predictions = (outputs >= 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
    
    accuracy = correct/total
    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f} - Accuracy: {accuracy*100:.2f}%")

Epoch 1 - Loss: 0.2656 - Accuracy: 88.95%
Epoch 2 - Loss: 0.2441 - Accuracy: 89.83%
Epoch 3 - Loss: 0.2237 - Accuracy: 90.89%
Epoch 4 - Loss: 0.1957 - Accuracy: 92.10%
Epoch 5 - Loss: 0.1798 - Accuracy: 92.80%
Epoch 6 - Loss: 0.1614 - Accuracy: 93.76%
Epoch 7 - Loss: 0.1427 - Accuracy: 94.54%
Epoch 8 - Loss: 0.1378 - Accuracy: 94.55%
Epoch 9 - Loss: 0.1118 - Accuracy: 95.75%
Epoch 10 - Loss: 0.1071 - Accuracy: 96.00%
